#### Data Curation and Population Extraction for emergency attendances with ECG screenings for prediction of acute cardiac diagnoses in hospital

In [9]:
import pandas as pd
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib.patches import Patch
from matplotlib.dates import DateFormatter
from datetime import timedelta, datetime
from tqdm import tqdm
import numpy as np
from scipy import stats, special
from tableone import TableOne

import os
import json
import re
import pprint
import missingno as msno
from statannotations.Annotator import Annotator
import warnings

pd.set_option('display.max_rows', None)

In [10]:
mimic_path = '../../data/MIMIC-IV/mimiciv/3.1/'
mimic_ed_path = '../../data/MIMIC-IV/mimic-iv-ed/3.1/'
mimic_ecg_path = '../../data/MIMIC-IV/mimic-iv-ecg/1.0/'
out_path = '../outputs/ext_data/'

path_to_local = '../../data/MIMIC-IV/config/'

In [11]:
### Helper-functions for extracting EHR data
def dataframe_from_csv(path, compression='gzip', header=0, index_col=0, chunksize=None):
    return pd.read_csv(path, compression=compression, header=header, index_col=index_col, chunksize=None)

#### Test ED metadata

In [ ]:
ed_md = dataframe_from_csv()

#### Curate hospital stays across ED, ICU and Hospital modules

In [4]:
admissions = dataframe_from_csv(os.path.join(mimic_path, 'hosp/admissions.csv.gz')).reset_index()
admissions_ed = admissions[admissions['edregtime'].notna()]
icu_stays = dataframe_from_csv(os.path.join(mimic_ed_path, 'icu/icustays.csv.gz')).reset_index()
ed_attendances = dataframe_from_csv(os.path.join(mimic_ed_path, 'ed/edstays.csv.gz')).reset_index()

In [5]:
ed_attendances.head()

,subject_id,hadm_id,stay_id,intime,outtime,gender,race,arrival_transport,disposition
0,10000032,22595853.0,33258284,2180-05-06 19:17:00,2180-05-06 23:30:00,F,WHITE,AMBULANCE,ADMITTED
1,10000032,22841357.0,38112554,2180-06-26 15:54:00,2180-06-26 21:31:00,F,WHITE,AMBULANCE,ADMITTED
2,10000032,25742920.0,35968195,2180-08-05 20:58:00,2180-08-06 01:44:00,F,WHITE,AMBULANCE,ADMITTED
3,10000032,29079034.0,32952584,2180-07-22 16:24:00,2180-07-23 05:54:00,F,WHITE,AMBULANCE,HOME
4,10000032,29079034.0,39399961,2180-07-23 05:54:00,2180-07-23 14:00:00,F,WHITE,AMBULANCE,ADMITTED


In [6]:
print(f'Admissions: {admissions_ed.shape}, ICU Stays: {icu_stays.shape}, ED Attendances: {ed_attendances.shape}')
print(f'Unique patients in Admissions: {admissions_ed["subject_id"].nunique()}, ICU Stays: {icu_stays["subject_id"].nunique()}, ED Attendances: {ed_attendances["subject_id"].nunique()}')

Admissions: (379240, 16), ICU Stays: (94458, 8), ED Attendances: (425087, 9)
Unique patients in Admissions: 182439, ICU Stays: 65366, ED Attendances: 205504


In [7]:
# Check if all hadm_id in icu_stays are present in admissions
all_ids_present = icu_stays['hadm_id'].isin(admissions['hadm_id']).all()
print("All hadm_id in icu_stays are present in admissions:", all_ids_present)

All hadm_id in icu_stays are present in admissions: True


In [8]:
ed_attendances['disposition'].value_counts()

disposition
HOME                           241632
ADMITTED                       158010
TRANSFER                         7025
LEFT WITHOUT BEING SEEN          6155
ELOPED                           5710
OTHER                            4297
LEFT AGAINST MEDICAL ADVICE      1881
EXPIRED                           377
Name: count, dtype: int64

In [9]:
ed_attendances.arrival_transport.value_counts()

arrival_transport
WALK IN       251849
AMBULANCE     155752
UNKNOWN        15352
OTHER           1266
HELICOPTER       868
Name: count, dtype: int64

In [10]:
# Filter ED stays for 'ADMITTED' disposition
admitted_ed_stays = ed_attendances[ed_attendances['disposition'] == 'ADMITTED']

# Count how many 'ADMITTED' stays have an associated hadm_id
total_admitted_stays = admitted_ed_stays.shape[0]
admitted_with_hadm_id = admitted_ed_stays['hadm_id'].notna().sum()

# Check how many of those hadm_ids in ed_attendances are also in admissions
hadm_ids_in_ed_stays = admitted_ed_stays['hadm_id'].dropna().unique()
hadm_ids_in_admissions = admissions_ed['hadm_id'].unique()

# Count the number of hadm_ids from admitted stays that are also in admissions
hadm_ids_in_both = sum(hadm_id in hadm_ids_in_admissions for hadm_id in hadm_ids_in_ed_stays)
hadm_ed_set_both = set(hadm_ids_in_ed_stays).intersection(set(hadm_ids_in_admissions))

# Output the results
print(f"Total 'ADMITTED' stays: {total_admitted_stays}")
print(f"'ADMITTED' stays with hadm_id: {admitted_with_hadm_id}")
print(f"hadm_ids from admitted stays that are in admissions: {hadm_ids_in_both}")

admissions_ed = admissions_ed[admissions_ed['hadm_id'].isin(hadm_ed_set_both)]
admitted_ed_stays = admitted_ed_stays[admitted_ed_stays['hadm_id'].isin(hadm_ed_set_both)]
print(f'Filtered Admissions: {admissions_ed.shape}, Filtered ADMITTED ED Stays: {admitted_ed_stays.shape}')

Total 'ADMITTED' stays: 158010
'ADMITTED' stays with hadm_id: 157626
hadm_ids from admitted stays that are in admissions: 157601
Filtered Admissions: (157601, 16), Filtered ADMITTED ED Stays: (157607, 9)


#### Link relevant lab tests

In [5]:
def read_d_labitems_table(mimic4_path):
    labitems = dataframe_from_csv(os.path.join(mimic4_path, 'hosp/d_labitems.csv.gz'))
    labitems.reset_index(inplace=True)
    return labitems[['itemid', 'label', 'category']]

In [12]:
lab_ids = read_d_labitems_table(mimic_path)

In [13]:
lab_ids.shape

(1650, 3)

In [14]:
lab_ids.category.value_counts()

category
Chemistry     800
Hematology    785
Blood Gas      65
Name: count, dtype: int64

In [40]:
lab_ids[lab_ids['label'].str.contains('react', case=False, na=False)]

,itemid,label,category
87,50889,C-Reactive Protein,Chemistry
1172,52082,Aspirin Reactivity,Hematology
1243,52154,P2Y12 Reaction Units,Hematology


In [37]:
troponin_ids = lab_ids[lab_ids['label'].str.contains('troponin', case=False, na=False)]
ck_ids = lab_ids[lab_ids['label'].str.contains('Creatine Kinase', case=False, na=False)]
haem_ids = lab_ids[lab_ids['label'].str.contains('hemoglobin', case=False, na=False)]
egfr_ids = lab_ids[lab_ids['label'].str.contains('gfr', case=False, na=False)]

In [38]:
egfr_ids

,itemid,label,category
118,50920,Estimated GFR (MDRD equation),Chemistry
886,51770,MDRDgfr,Chemistry
1116,52026,Estimated GFR (MDRD equation),Blood Gas
1620,53161,Estimated GFR (CKD- EPI Refit),Chemistry
1639,53180,Estimated GFR (CKD- EPI 2021),Blood Gas


In [34]:
haem_ids

,itemid,label,category
4,50805,Carboxyhemoglobin,Blood Gas
9,50811,Hemoglobin,Blood Gas
12,50814,Methemoglobin,Blood Gas
50,50852,% Hemoglobin A1c,Chemistry
53,50855,Absolute Hemoglobin,Chemistry
397,51212,Fetal Hemoglobin,Hematology
407,51222,Hemoglobin,Hematology
408,51223,Hemoglobin A2,Hematology
409,51224,Hemoglobin C,Hematology
410,51225,Hemoglobin F,Hematology


In [25]:
ck_ids

,itemid,label,category
108,50910,Creatine Kinase (CK),Chemistry
109,50911,"Creatine Kinase, MB Isoenzyme",Chemistry
717,51594,"Creatine Kinase, Isoenzyme BB",Chemistry
718,51595,"Creatine Kinase, Isoenzyme MB",Chemistry
719,51596,"Creatine Kinase, Isoenzyme MM",Chemistry


In [26]:
troponin_ids

,itemid,label,category
198,51002,Troponin I,Chemistry
199,51003,Troponin T,Chemistry
1528,52642,Troponin I,Chemistry


#### Read medications table

In [6]:
presc_data = dataframe_from_csv(os.path.join(mimic_path, 'hosp/emar.csv.gz')).reset_index()

In [ ]:
def get_generic_drugs(mapping, df):
    """Takes NDC product table and prescriptions dataframe; adds column with NDC table's corresponding generic name"""

    def brand_to_generic(ndc):
        # We only want the first 2 sections of the NDC code: xxxx-xxxx-xx
        matches = list(re.finditer(r"-", ndc))
        if len(matches) > 1:
            ndc = ndc[:matches[1].start()]
        try:
            return mapping.loc[mapping.PRODUCTNDC == ndc].NONPROPRIETARYNAME.iloc[0]
        except:
            print("Error: ", ndc)
            return np.nan

    df['generic_drug_name'] = df['ndc'].apply(brand_to_generic)

In [7]:
presc_data.shape, presc_data.columns

((42808593, 12),
 Index(['subject_id', 'hadm_id', 'emar_id', 'emar_seq', 'poe_id', 'pharmacy_id',
        'enter_provider_id', 'charttime', 'medication', 'event_txt',
        'scheduletime', 'storetime'],
       dtype='object'))

In [8]:
presc_data[presc_data.medication.str.contains('aspirin', case=False, na=False)]

KeyboardInterrupt: 

In [45]:
presc_data.drug_type.value_counts()

drug_type
MAIN        16791812
BASE         3471112
ADDITIVE       29687
Name: count, dtype: int64

In [ ]:
lab_ids = dataframe_from_csv(os.path.join(mimic_path, 'hosp/d_labitems.csv.gz'), compression=None, header=0, index_col=None)
lab_ids.shape

In [ ]:
#### HS Troponin cleaning and extraction

# Filter lab_ids for rows where the label contains 'troponin'
troponin_ids = lab_ids[lab_ids['label'].str.contains('troponin', case=False, na=False)]

# Extract the itemid values for these rows
troponin_itemids = troponin_ids['itemid'].unique()

# Filter the lab_ids DataFrame to keep only rows where itemid is 51003
lab_troponin_t = labs[labs['itemid'] == 51003]
print(lab_troponin_t.shape)

# Fill NaN values in valuenum where comments start with '<' or 'LESS' with 0.01
lab_troponin_t.loc[
    lab_troponin_t['valuenum'].isna() & lab_troponin_t['comments'].str.startswith(('<', 'LESS')),
    'valuenum'
] = 0.01

# Fill NaN values in valuenum where comments start with '>' or 'GREATER' with 25
lab_troponin_t.loc[
    lab_troponin_t['valuenum'].isna() & lab_troponin_t['comments'].str.startswith(('>', 'GREATER')),
    'valuenum'
] = 25

# Display the updated DataFrame or the first few rows to confirm
lab_troponin_t.shape

# Check how many NaN values are left in valuenum
nans_left = lab_troponin_t['valuenum'].isna().sum()
print(f"Number of NaN values left in valuenum: {nans_left}")

# Drop rows where valuenum is still NaN
lab_troponin_t_cleaned = lab_troponin_t.dropna(subset=['valuenum'])

# Check the shape of the cleaned DataFrame to confirm
print(f"Shape of DataFrame after dropping NaNs: {lab_troponin_t_cleaned.shape}")

In [ ]:
### Creatine Kinase

# Filter lab_ids for rows where the label contains 'Creatine Kinase'
ck_ids = lab_ids[lab_ids['label'].str.contains('Creatine Kinase', case=False, na=False)]

# Extract the itemid values for these rows
ck_itemids = ck_ids['itemid'].unique()

# Filter the lab_ids DataFrame to keep only rows where itemid is 51002
lab_ck = lab[lab['itemid'] == 50910]
lab_ck.shape